### **IMPORT THINGS NEEDED**

In [1]:
!pip install pytorchcv
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
from torchvision import models
import torchvision.utils
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from pytorchcv.model_provider import get_model as ptcv_get_model
from PIL import Image
import os

     |████████████████████████████████| 532 kB 5.0 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

### **Loading Data CIFAR100**

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), 
])

In [3]:
#! wget -O cifar-normal-test.zip 'https://www.dropbox.com/s/cj68qxfws3g9c39/CIFAR-100-test.zip?dl=1'
#! wget -O adversarial-test.zip 'https://www.dropbox.com/s/g9eeyrwq2nbuxfr/adv_imgs_vgg16_ifgsm.zip?dl=1'
! wget https://www.dropbox.com/sh/13aqnou6hnyl2lq/AAC0enwbI5LgshKLT5zgQDFLa?dl=1 --content-disposition

--2022-05-12 02:55:44--  https://www.dropbox.com/sh/13aqnou6hnyl2lq/AAC0enwbI5LgshKLT5zgQDFLa?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/dl/13aqnou6hnyl2lq/AAC0enwbI5LgshKLT5zgQDFLa [following]
--2022-05-12 02:55:44--  https://www.dropbox.com/sh/dl/13aqnou6hnyl2lq/AAC0enwbI5LgshKLT5zgQDFLa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9ec08b043e93f22d00f4ab39ab.dl.dropboxusercontent.com/zip_download_get/BIflAVu3XsSQpXqW5sZdEkrS9hkIwS4p_zUiReH2zjwljAew9W5rYBGNsiW8SNHcRodZd33GdF-yS6YZzcXpkuq7Wi4CCz5sQPzR30HkU2-65g?dl=1# [following]
--2022-05-12 02:55:45--  https://uc9ec08b043e93f22d00f4ab39ab.dl.dropboxusercontent.com/zip_download_get/BIflAVu3XsSQpXqW5sZdEkrS9hkIwS4p_zUiReH2zjwljAew9W5rYBGNsiW8SNHcRodZd33GdF-yS

In [4]:
#!unzip cifar-normal-test.zip
#!unzip adversarial-test.zip
!unzip adv_imgs_8.zip

Archive:  adv_imgs_8.zip
mapname:  conversion of  failed
 extracting: adv_imgs_dithering_vgg16_fgsm.zip  
 extracting: adv_imgs_dithering_vgg16_ifgsm.zip  
 extracting: adv_imgs_dithering_resnet_fgsm.zip  
 extracting: adv_imgs _dithering_resnet_ifgsm.zip  
 extracting: adv_imgs_errdithering_vgg16_fgsm.zip  
 extracting: adv_imgs_errdithering_vgg16_ifgsm.zip  
 extracting: adv_imgs_ErrorDithering_resnet_fgsm.zip  
 extracting: adv_imgs_ErrorDithering_resnet_ifgsm.zip  


In [26]:
!unzip adv_imgs_errdithering_vgg16_fgsm.zip

串流輸出內容已截斷至最後 5000 行。
 extracting: adv_imgs/5_307.png      
 extracting: adv_imgs/9_371.png      
 extracting: adv_imgs/10_158.png     
 extracting: adv_imgs/5_452.png      
 extracting: adv_imgs/17_243.png     
 extracting: adv_imgs/4_23.png       
 extracting: adv_imgs/17_248.png     
 extracting: adv_imgs/14_69.png      
 extracting: adv_imgs/13_72.png      
 extracting: adv_imgs/4_38.png       
 extracting: adv_imgs/5_489.png      
 extracting: adv_imgs/1_102.png      
 extracting: adv_imgs/6_75.png       
 extracting: adv_imgs/11_211.png     
 extracting: adv_imgs/2_68.png       
 extracting: adv_imgs/4_108.png      
 extracting: adv_imgs/1_194.png      
 extracting: adv_imgs/7_351.png      
 extracting: adv_imgs/19_409.png     
 extracting: adv_imgs/6_55.png       
 extracting: adv_imgs/8_488.png      
 extracting: adv_imgs/10_405.png     
 extracting: adv_imgs/13_215.png     
 extracting: adv_imgs/17_153.png     
 extracting: adv_imgs/16_164.png     
 extracting: adv_imgs/7_307.p

In [6]:
class FoodDataset(Dataset):

    def __init__(self, path, tfm=transform, files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".png")])
        if files != None:
            self.files = files
        print(f"One {path} sample",self.files[0])
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [27]:
data = FoodDataset('adv_imgs')
#print(data.target)
loader = Data.DataLoader(data, batch_size=25, shuffle=False, num_workers=0, pin_memory=True)
print(len(loader), len(loader.dataset))

One adv_imgs sample adv_imgs/0_0.png
400 10000


In [18]:
adv_name = []
for fname in data.files:
    name = fname.split('/')[-1]
    adv_name.append(name)
print(adv_name)

['0_0.png', '0_1.png', '0_10.png', '0_100.png', '0_101.png', '0_102.png', '0_103.png', '0_104.png', '0_105.png', '0_106.png', '0_107.png', '0_108.png', '0_109.png', '0_11.png', '0_110.png', '0_111.png', '0_112.png', '0_113.png', '0_114.png', '0_115.png', '0_116.png', '0_117.png', '0_118.png', '0_119.png', '0_12.png', '0_120.png', '0_121.png', '0_122.png', '0_123.png', '0_124.png', '0_125.png', '0_126.png', '0_127.png', '0_128.png', '0_129.png', '0_13.png', '0_130.png', '0_131.png', '0_132.png', '0_133.png', '0_134.png', '0_135.png', '0_136.png', '0_137.png', '0_138.png', '0_139.png', '0_14.png', '0_140.png', '0_141.png', '0_142.png', '0_143.png', '0_144.png', '0_145.png', '0_146.png', '0_147.png', '0_148.png', '0_149.png', '0_15.png', '0_150.png', '0_151.png', '0_152.png', '0_153.png', '0_154.png', '0_155.png', '0_156.png', '0_157.png', '0_158.png', '0_159.png', '0_16.png', '0_160.png', '0_161.png', '0_162.png', '0_163.png', '0_164.png', '0_165.png', '0_166.png', '0_167.png', '0_168.pn

### **Loading Model**

In [9]:
class vgg(nn.Module):
    def __init__(self):
        super(vgg, self).__init__()
        self.model = ptcv_get_model("resnet1202_cifar100", pretrained=True)
        self.fc = nn.Sequential(
            nn.Linear(100, 20),
           # nn.LeakyReLU(0.1),
           # nn.Linear(512, 20),
        )

    def forward(self, x):
        out = self.model(x)
        return self.fc(out)

In [9]:
import torchvision.models as models
class scratch_model(nn.Module):
    def __init__(self):
        super(scratch_model, self).__init__()
        self.model = models.vgg16(pretrained=True)
        self.fc = nn.Sequential(
            nn.Linear(1000, 512),
            nn.LeakyReLU(0.1),
            nn.Linear(512, 20),
        )

    def forward(self, x):
        out = self.model(x)
        return self.fc(out)

In [10]:
!wget -O model.zip https://www.dropbox.com/s/v6c756rjb24155d/dip_models.zip?dl=1

--2022-05-12 02:58:27--  https://www.dropbox.com/s/v6c756rjb24155d/dip_models.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/v6c756rjb24155d/dip_models.zip [following]
--2022-05-12 02:58:27--  https://www.dropbox.com/s/dl/v6c756rjb24155d/dip_models.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5c0959d4e6df3a770f4a5c0adf.dl.dropboxusercontent.com/cd/0/get/BlHJeZ7sBrbl1AoIZX1alaTEQgtCO_bLGrkeJVgTm5a0KU9jnzIwMty2amelpaOARv5t6y6jzLKSILRoOZKDwNeoo1v3no3YwLMjUpKLaCLlIfXzX-9d8vD7DGmgRBDqvDPtcpuM1oqdK5ukaIpjqcx76hCe8BodtTky-8iljDK1GQpGEAzoqQ4-bHFB8BStIJ0/file?dl=1# [following]
--2022-05-12 02:58:27--  https://uc5c0959d4e6df3a770f4a5c0adf.dl.dropboxusercontent.com/cd/0/get/BlHJeZ7sBrbl1AoIZX1alaTEQgtCO_bLGrkeJVg

In [11]:
!unzip model.zip

Archive:  model.zip
  inflating: dithering_resnet.ckpt   
  inflating: ErrorDithering_vgg16.ckpt  
  inflating: dithering_vgg16.ckpt    
  inflating: ErrorDithering_resnet.ckpt  


In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = scratch_model().to(device)
model.load_state_dict(torch.load('ErrorDithering_vgg16.ckpt'))

<All keys matched successfully>

In [13]:
mean = torch.tensor([0.485, 0.456, 0.406]).to(device).view(3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).to(device).view(3, 1, 1)

In [14]:
!nvidia-smi

Thu May 12 02:59:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W /  70W |   2316MiB / 15109MiB |     22%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [28]:
model.eval()
test_accs = []
for images, labels in loader:
    images = images.to(device)
    outputs = model(images)
    _, pre = torch.max(outputs.data, 1)
    acc = (pre.cpu() == labels).float().mean()
    test_accs.append(acc)
    print(acc)

test_acc = sum(test_accs) / len(test_accs)
print('Accuracy of test text: %f %%' % (100 * test_acc))

tensor(0.0400)
tensor(0.)
tensor(0.)
tensor(0.0800)
tensor(0.0400)
tensor(0.0800)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.0400)
tensor(0.0400)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.0800)
tensor(0.)
tensor(0.0400)
tensor(0.)
tensor(0.)
tensor(0.0400)
tensor(0.0800)
tensor(0.0800)
tensor(0.0800)
tensor(0.)
tensor(0.1600)
tensor(0.0400)
tensor(0.0800)
tensor(0.1200)
tensor(0.1200)
tensor(0.0800)
tensor(0.0800)
tensor(0.1200)
tensor(0.1200)
tensor(0.1200)
tensor(0.0800)
tensor(0.0800)
tensor(0.0400)
tensor(0.0400)
tensor(0.0800)
tensor(0.)
tensor(0.1200)
tensor(0.1200)
tensor(0.1600)
tensor(0.)
tensor(0.0800)
tensor(0.1200)
tensor(0.0400)
tensor(0.0800)
tensor(0.1200)
tensor(0.0800)
tensor(0.0800)
tensor(0.0800)
tensor(0.0400)
tensor(0.)
tensor(0.1600)
tensor(0.0400)
tensor(0.0800)
tensor(0.0400)
tensor(0.1200)
tensor(0.0400)
tensor(0.0800)
tensor(0.0800)
tensor(0.0800)
tensor(0.1600)
tensor(0.1600)
tensor(0.2000)
tensor(0.0400)
tensor(0.0800)
tensor(0.0400)
tensor(0.0400)
tensor(0.2

### **Making Adversarial Example**

In [15]:
def ifgsm(model, x, y, epsilon=8/255/std, alpha=2/255/std, num_iter=20):
    x, y = x.to(device), y.to(device)
    x_adv = x
    loss_fn = nn.CrossEntropyLoss()
    # write a loop of num_iter to represent the iterative times
    for i in range(num_iter):
        # x_adv = fgsm(model, x_adv, y, loss_fn, alpha) # call fgsm with (epsilon = alpha) to obtain new x_adv
        x_adv = x_adv.detach().clone()
        x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
        loss = loss_fn(model(x_adv), y) # calculate loss
        loss.backward() # calculate gradient
        # fgsm: use gradient ascent on x_adv to maximize loss
        grad = x_adv.grad.detach()
        x_adv = x_adv + alpha * grad.sign()

        x_adv = torch.max(torch.min(x_adv, x+epsilon), x-epsilon) # clip new x_adv back to [x-epsilon, x+epsilon]
    return x_adv

def fgsm(model, x, y, epsilon=8/255/std):
    x, y = x.to(device), y.to(device)
    loss_fn = nn.CrossEntropyLoss()
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    loss = loss_fn(model(x_adv), y) # calculate loss
    loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    grad = x_adv.grad.detach()
    x_adv = x_adv + epsilon * grad.sign()
    return x_adv

In [31]:
! rm -rf adv_img*

In [32]:
print("Attack Image & Predicted Label")
model.eval()
output_dir = 'adv_imgs'

if not os.path.exists(output_dir):
  os.makedirs(output_dir)
train_acc = 0.
i = 0
for images, labels in loader:
    images = fgsm(model, images, labels)
    labels = labels.to(device)
    outputs = model(images)
    train_acc += (outputs.argmax(dim=1) == labels).sum().item()
    images = ((images) * std + mean).clamp(0, 1)
    images = (images * 255).clamp(0, 255)
    images = images.detach().cpu().data.numpy().round()
    images = images.transpose((0, 2, 3, 1))
    adv_examples = images if i == 0 else np.r_[adv_examples, images]
    i += 1
    print(i)
print('Accuracy of test text: %f %%' % (100 * train_acc / 10000))

Attack Image & Predicted Label
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
2

In [33]:
def create_dir(adv_dir, adv_examples, adv_names):
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8)) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

In [34]:
create_dir(output_dir, adv_examples, adv_name)

In [35]:
!zip -r adv_imgs.zip adv_imgs

串流輸出內容已截斷至最後 5000 行。
  adding: adv_imgs/5_307.png (stored 0%)
  adding: adv_imgs/9_371.png (stored 0%)
  adding: adv_imgs/10_158.png (stored 0%)
  adding: adv_imgs/5_452.png (stored 0%)
  adding: adv_imgs/17_243.png (stored 0%)
  adding: adv_imgs/4_23.png (stored 0%)
  adding: adv_imgs/17_248.png (stored 0%)
  adding: adv_imgs/14_69.png (stored 0%)
  adding: adv_imgs/13_72.png (stored 0%)
  adding: adv_imgs/4_38.png (stored 0%)
  adding: adv_imgs/5_489.png (stored 0%)
  adding: adv_imgs/1_102.png (stored 0%)
  adding: adv_imgs/6_75.png (stored 0%)
  adding: adv_imgs/11_211.png (stored 0%)
  adding: adv_imgs/2_68.png (stored 0%)
  adding: adv_imgs/4_108.png (stored 0%)
  adding: adv_imgs/1_194.png (stored 0%)
  adding: adv_imgs/7_351.png (stored 0%)
  adding: adv_imgs/19_409.png (stored 0%)
  adding: adv_imgs/6_55.png (stored 0%)
  adding: adv_imgs/8_488.png (stored 0%)
  adding: adv_imgs/10_405.png (stored 0%)
  adding: adv_imgs/13_215.png (stored 0%)
  adding: adv_imgs/17_153.png (sto

In [36]:
from google.colab import files
files.download('adv_imgs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Varify epsilon**

In [37]:
original_dir = 'CIFAR-100-test'
adv_dir = 'adv_imgs'
for i in range(20):
  for j in range(500):
    orig_img = Image.open(os.path.join(original_dir, '{}_{}.png'.format(i, j)))
    orig_img = np.intc(orig_img)
    adv_img = Image.open(os.path.join(adv_dir, '{}_{}.png'.format(i, j)))
    adv_img = np.intc(adv_img)
    dis = np.abs(orig_img-adv_img)
    if np.max(dis) > 8:
      print("error!")